In [ ]:
import torch
import cv2
import numpy as np
from ultralytics import YOLO

live_model = YOLO('yolov8s.pt')
video_model = YOLO('ball_track_model/last_t2.pt')

def detect_ball_with_yolo(frame, model):
    blurred_frame = cv2.GaussianBlur(frame, (5, 5), 0)
    hsv_frame = cv2.cvtColor(blurred_frame, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([20, 100, 100])
    upper_yellow = np.array([30, 255, 255])
    mask = cv2.inRange(hsv_frame, lower_yellow, upper_yellow)
    filtered_frame = cv2.bitwise_and(blurred_frame, blurred_frame, mask=mask)
    results = model(filtered_frame)
    boxes = results[0].boxes.cpu().numpy()
    ball_positions = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        class_id = int(box.cls[0])
        if class_id == 32:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, "Tennis Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            ball_positions.append((x1, y1, x2, y2))
    return frame, ball_positions

def detect_ball_in_video_frame(frame, model):
    results = model(frame)
    boxes = results[0].boxes
    ball_positions = []
    for box in boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
        class_id = int(box.cls[0])
        if class_id == 0:
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, f"Tennis Ball", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            ball_positions.append((x1, y1, x2, y2))
    return frame, ball_positions

def apply_roi(frame):
    height, width = frame.shape[:2]
    x_start = int(width * 0.1)
    y_start = int(height * 0.17)
    x_end = int(width * 0.9)
    y_end = int(height * 0.9)
    mask = np.zeros_like(frame)
    mask[y_start:y_end, x_start:x_end] = frame[y_start:y_end, x_start:x_end]
    return mask

def fill_edges(edges):
    kernel = np.ones((5, 5), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=2)
    closed_edges = cv2.morphologyEx(dilated_edges, cv2.MORPH_CLOSE, kernel)
    return closed_edges

def detect_court_lines(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(gray, 50, 150)
    return edges

def find_squares(filled_edges):
    contours, _ = cv2.findContours(filled_edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    squares = []
    for contour in contours:
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        area = cv2.contourArea(contour)
        if len(approx) == 4 and area > 500:
            squares.append(approx)
    return squares

def overlay_squares_on_frame(frame, squares, ball_positions, prev_ball_positions):
    for square in squares:
        polygon = square.reshape((-1, 2))
        ball_hit_ground = False
        for idx, (x1, y1, x2, y2) in enumerate(ball_positions):
            if idx < len(prev_ball_positions):
                prev_x1, prev_y1, prev_x2, prev_y2 = prev_ball_positions[idx]
                current_center = ((x1 + x2) // 2, (y1 + y2) // 2)
                prev_center = ((prev_x1 + prev_x2) // 2, (prev_y1 + prev_y2) // 2)
                if prev_center[1] < current_center[1]:
                    moving_down = True
                else:
                    moving_down = False
                if moving_down == False and cv2.pointPolygonTest(polygon, current_center, False) >= 0:
                    ball_hit_ground = True
                    break
        if ball_hit_ground:
            color = (0, 0, 255)
        else:
            color = (0, 255, 0)
        cv2.drawContours(frame, [square], 0, color, 3)
    return frame

def main():
    use_camera = False
    prev_ball_positions = []

    if use_camera:
        cap = cv2.VideoCapture(0) # iPhone(0) MacBook(1)
        model = live_model
    else:
        cap = cv2.VideoCapture('Zverev.mp4') # Dan.mp4 or Zverev.mp4
        model = video_model
        print("Class names in custom model:", model.names)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if use_camera:
            roi_frame = frame.copy()
            result_ball, ball_positions = detect_ball_with_yolo(roi_frame, model)
        else:
            roi_frame = apply_roi(frame.copy())
            result_ball, ball_positions = detect_ball_in_video_frame(frame.copy(), model)

        edges = detect_court_lines(roi_frame)
        filled_edges = fill_edges(edges)
        squares = find_squares(filled_edges)

        output_frame = overlay_squares_on_frame(result_ball, squares, ball_positions, prev_ball_positions)
        prev_ball_positions = ball_positions.copy()

        cv2.imshow('Tennis Ball and Court Detection', output_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
